<a href="https://colab.research.google.com/github/ouafighizlene21/Twitter-Disinformation-Study/blob/main/21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


In [45]:
import pandas as pd

df = pd.read_csv('DetectDepressionInTwitterPosts/depressive_tweets_processed.csv')
print(df.head())


  989292962323615744|2018-04-25|23:59:57|Eastern Standard Time|whosalli|The lack of this understanding is a small but significant part of what causes anxiety & depression to both feel so incredibly lonely. It's soooo easy to compare. It's so easy to invalidate ourselves because of that.|1|0|3|  \
0  989292959844663296|2018-04-25|23:59:56|Eastern...                                                                                                                                                                                                                                                     
1  989292951716155392|2018-04-25|23:59:54|Eastern...                                                                                                                                                                                                                                                     
2  989292873664393218|2018-04-25|23:59:35|Eastern...                                                      

In [47]:
with open('DetectDepressionInTwitterPosts/depressive_tweets_processed.csv', 'r', encoding='utf-8') as f:
    for _ in range(10):
        print(f.readline())


989292962323615744|2018-04-25|23:59:57|Eastern Standard Time|whosalli|The lack of this understanding is a small but significant part of what causes anxiety & depression to both feel so incredibly lonely. It's soooo easy to compare. It's so easy to invalidate ourselves because of that.|1|0|3|,,,,,,,,,,,,,,,,,,,,,,,,,

989292959844663296|2018-04-25|23:59:56|Eastern Standard Time|estermnunes|i just told my parents about my depression and it's so hard to get gen x people to understand that this is not something that i can control all the time or just cure with a walk or by keeping my mind busy|1|0|2|,,,,,,,,,,,,,,,,,,,,,,,,,

989292951716155392|2018-04-25|23:59:54|Eastern Standard Time|TheAlphaAries|depression is something i don't speak about even going through it because it's also such a double edged sword. i love every race. even if white people had done so much i can't hate them all. my grandma is legit white! how tf can i hate, i do dislike people though.|0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,

In [48]:
import pandas as pd

# Lire le fichier avec un nombre de colonnes élevé
df = pd.read_csv(
    'DetectDepressionInTwitterPosts/depressive_tweets_processed.csv',
    sep='|',
    header=None,
    engine='python',
    on_bad_lines='skip',  # Ignore les lignes corrompues
    names=[
        'id', 'date', 'time', 'timezone', 'username',
        'text', 'reply_count', 'retweet_count', 'like_count',
        *['extra_' + str(i) for i in range(25)]  # Pour capturer les NaN/virgules
    ]
)

# Afficher les premières lignes
df.head()


,id,date,time,timezone,username,text,reply_count,retweet_count,like_count,extra_0,...,extra_15,extra_16,extra_17,extra_18,extra_19,extra_20,extra_21,extra_22,extra_23,extra_24
0,989292962323615744,2018-04-25,23:59:57,Eastern Standard Time,whosalli,The lack of this understanding is a small but ...,1,0,3,",,,,,,,,,,,,,,,,,,,,,,,,,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,989292959844663296,2018-04-25,23:59:56,Eastern Standard Time,estermnunes,i just told my parents about my depression and...,1,0,2,",,,,,,,,,,,,,,,,,,,,,,,,,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,989292951716155392,2018-04-25,23:59:54,Eastern Standard Time,TheAlphaAries,depression is something i don't speak about ev...,0,0,0,",,,,,,,,,,,,,,,,,,,,,,,,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,989292873664393218,2018-04-25,23:59:35,Eastern Standard Time,_ojhodgson,Made myself a tortilla filled with pb&j. My de...,1,0,0,",,,,,,,,,,,,,,,,,,,,,,,,,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,989292856119472128,2018-04-25,23:59:31,Eastern Standard Time,DMiller96371630,@WorldofOutlaws I am gonna need depression med...,0,0,0,",,,,,,,,,,,,,,,,,,,,,,,,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# Ne garder que les colonnes importantes
df_clean = df[['id', 'date', 'time', 'timezone', 'username', 'text', 'reply_count', 'retweet_count', 'like_count']]

# Afficher
df_clean.head()


,id,date,time,timezone,username,text,reply_count,retweet_count,like_count
0,989292962323615744,2018-04-25,23:59:57,Eastern Standard Time,whosalli,The lack of this understanding is a small but ...,1,0,3
1,989292959844663296,2018-04-25,23:59:56,Eastern Standard Time,estermnunes,i just told my parents about my depression and...,1,0,2
2,989292951716155392,2018-04-25,23:59:54,Eastern Standard Time,TheAlphaAries,depression is something i don't speak about ev...,0,0,0
3,989292873664393218,2018-04-25,23:59:35,Eastern Standard Time,_ojhodgson,Made myself a tortilla filled with pb&j. My de...,1,0,0
4,989292856119472128,2018-04-25,23:59:31,Eastern Standard Time,DMiller96371630,@WorldofOutlaws I am gonna need depression med...,0,0,0


In [50]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [51]:
# Télécharger les ressources NLTK une seule fois
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [52]:
# Préparer le lemmatizer et les stopwords anglais
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [53]:
# Fonction de nettoyage
def preprocess_text(text):
    if pd.isna(text):
        return ""

    text = text.lower()  # lowercase
    text = re.sub(r'@\w+', '', text)  # remove mentions
    text = re.sub(r'#\w+', '', text)  # remove hashtags
    text = re.sub(r'http\S+|www\S+', '', text)  # remove URLs
    text = re.sub(r'[^a-z\s]', '', text)  # remove punctuation/numbers
    tokens = text.split()  # tokenize
    tokens = [word for word in tokens if word not in stop_words]  # remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # lemmatize
    return " ".join(tokens)

In [54]:
# Appliquer à la colonne texte (ajuste le nom si nécessaire)
df['clean_text'] = df[5].apply(preprocess_text)  # ou 'text' si tu as renommé


KeyError: 5

In [ ]:
# Afficher les résultats nettoyés
print(df[['clean_text']].head())

In [56]:
df.iloc[:, 5]  # colonne à l'index 5 (la 6e colonne, car on commence à 0)


,text
0,The lack of this understanding is a small but ...
1,i just told my parents about my depression and...
2,depression is something i don't speak about ev...
3,Made myself a tortilla filled with pb&j. My de...
4,@WorldofOutlaws I am gonna need depression med...
...,...
3447,Many sufferers of depression aren't sad; they ...
3448,No Depression by G Herbo is my mood from now o...
3449,What do you do when depression succumbs the br...
3450,Ketamine Nasal Spray Shows Promise Against Dep...


In [57]:
print(df.columns)


Index(['id', 'date', 'time', 'timezone', 'username', 'text', 'reply_count',
       'retweet_count', 'like_count', 'extra_0', 'extra_1', 'extra_2',
       'extra_3', 'extra_4', 'extra_5', 'extra_6', 'extra_7', 'extra_8',
       'extra_9', 'extra_10', 'extra_11', 'extra_12', 'extra_13', 'extra_14',
       'extra_15', 'extra_16', 'extra_17', 'extra_18', 'extra_19', 'extra_20',
       'extra_21', 'extra_22', 'extra_23', 'extra_24'],
      dtype='object')


In [58]:
# Supprimer les colonnes inutiles
colonnes_a_supprimer = [f'extra_{i}' for i in range(25)]
df.drop(columns=colonnes_a_supprimer, inplace=True)

# Vérification des colonnes restantes
df.columns


Index(['id', 'date', 'time', 'timezone', 'username', 'text', 'reply_count',
       'retweet_count', 'like_count'],
      dtype='object')